In [1]:
from run_risk import run_portfolio, run_stock, run_what_if, run_min_variance, run_max_return

In [2]:
# === OPENAI API Setup ===
import openai
from io import StringIO
from contextlib import redirect_stdout
from dotenv import load_dotenv
import os

load_dotenv()  # This loads variables from .env into the environment

# Now access your keys like this
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [3]:
def interpret_portfolio_risk(diagnostics_text: str) -> str:
    """
    Sends raw printed diagnostics to GPT for layman interpretation.
    """
    user_prompt = (
        "You are a professional risk analyst at a hedge fund.\n"
        "I want you to help evaluate my portfolio. I will give you details of the portfolio's risk metrics.\n"
        "I want you to help interpret them for me and communicate with me in simple language.\n\n"
        f"{diagnostics_text}"
    )

    response = client.chat.completions.create(
        model="gpt-4.1",  # or "gpt-4o-mini" if desired
        messages=[
            {"role": "system", "content": "You are a portfolio risk analysis expert."},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=2000,
        temperature=0.5
    )

    return response.choices[0].message.content.strip()

In [4]:
# Redirect printed output from the portfolio run
f = StringIO()
with redirect_stdout(f):
    run_portfolio("portfolio.yaml")

raw_output = f.getvalue()

In [5]:
# Run GPT-based interpretation
interpretation = interpret_portfolio_risk(raw_output)

print("\n=== GPT Portfolio Interpretation ===\n")
print(interpretation)


=== GPT Portfolio Interpretation ===

Great, thanks for sharing such detailed risk metrics! Let’s break down what your portfolio is telling us, step by step, in plain English.

---

## 1. **Portfolio Structure & Concentration**

- **Weights:** Your largest position is **AT.L** (28%), with the rest between 10% and 17%. **SHV** is your smallest (10%), likely a cash or ultra-short bond proxy.
- **Herfindahl Index (0.185):** This measures concentration. Values closer to 0 mean more diversification. Your portfolio is moderately concentrated, mainly due to AT.L.
- **Leverage:** Not leveraged (gross = net = 1.0).

---

## 2. **Risk & Return**

- **Annual Volatility:** **21.65%** – this is a moderate-to-high risk level, typical for equity-heavy portfolios.
- **Expected Returns:** Highest for **AT.L** (25%), then **NVDA** (20%), **PCTY** (17%), **MSCI** (16%), **TW** (15%).
- **Recent Performance:** Returns are volatile, with both strong gains and steep losses (e.g., -14% in April 2022).

---
